In [1]:
import pandas as pd
import seaborn as sns
from scipy import stats
import numpy as np

In [2]:
path='../../data/data2/raw_2/deliveries.csv'

In [4]:
deliveries = pd.read_csv(path,encoding='latin-1')

In [6]:
deliveries.head()

,id,route_id,state,_status,items,total_size,pickupLat,pickupLng,deliveryLat,deliveryLng,...,delivery_arrival_dt,delivery_effective_dt,cost,final_price,distance,shipper_id,distance_charged,distance_route,total_package_price,vehicle_capacity
0,t-amabrt-1338381,r-shiinc-1011719,7,completed,"[{""id"":""SFYBR003431115"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9268,-44.0977,...,2022-01-04 11:26:49,2022-01-04 11:29:14,2.94,4.87,11.6,266665.0,0.5,0.5,120.62,2
1,t-amabrt-1338382,r-shiinc-1015749,7,completed,"[{""id"":""SFYBR003431932"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9304,-43.9361,...,2022-01-05 20:07:03,2022-01-05 20:10:44,1.95,7.71,12.0,292684.0,0.5,0.5,272.90,2
2,t-amabrt-1338383,r-shiinc-1013679,7,completed,"[{""id"":""SFYBR003431610"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8080,-43.9787,...,2022-01-05 18:53:59,2022-01-05 18:54:22,4.05,4.87,24.6,219267.0,0.3,0.3,99.01,2
3,t-amabrt-1338384,r-shiinc-1011705,7,completed,"[{""id"":""SFYBR003438164"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.8146,-43.9768,...,2022-01-03 15:50:15,2022-01-03 21:31:53,2.99,4.87,23.3,324993.0,0.3,0.3,69.90,2
4,t-amabrt-1338385,r-shiinc-1011717,7,completed,"[{""id"":""SFYBR003431094"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.9729,-43.9701,...,2022-01-03 15:46:43,2022-01-03 15:47:42,3.19,4.80,10.5,244005.0,0.3,0.3,20.62,2


In [12]:
deliveries.shape

(594268, 26)

In [7]:
routes_ids_completed=deliveries.route_id.loc[deliveries.state == 7].unique()

In [8]:
routes_totally_complete=[]
for i in routes_ids_completed:
  routes_status=deliveries._status[deliveries.route_id==i].unique()
  if routes_status.size==1 and routes_status[0]=='completed':
    routes_totally_complete.append(i)

In [10]:
deliveries = deliveries.loc[deliveries.route_id.isin( routes_totally_complete)]

In [11]:
deliveries.shape

(594268, 26)

In [13]:
date_fields = [
  "pickup_dt", "max_pickup_dt", "pickup_effective_dt", 
  "delivery_dt", "max_delivery_dt", "delivery_effective_dt"
]
for field in date_fields:
  deliveries = deliveries.loc[deliveries[field] != "0000-00-00 00:00:00"]
  deliveries[field] = pd.to_datetime(deliveries[field])
deliveries.dropna(subset=date_fields, inplace=True)
deliveries.shape

(594268, 26)

In [14]:
deliveries.isna().sum()

id                            0
route_id                      0
state                         0
_status                       0
items                         0
total_size                    0
pickupLat                     0
pickupLng                     0
deliveryLat                   0
deliveryLng                   0
pickup_dt                     0
max_pickup_dt                 0
pickup_arrival_dt             5
pickup_effective_dt           0
delivery_dt                   0
max_delivery_dt               0
delivery_arrival_dt         357
delivery_effective_dt         0
cost                          0
final_price                   0
distance                      0
shipper_id                   31
distance_charged              0
distance_route                0
total_package_price      193415
vehicle_capacity              0
dtype: int64

In [15]:
path_steps='../../data/data2/raw_2/steps-001.csv'

In [16]:
steps = pd.read_csv(path_steps)

In [17]:
steps.shape

(51848524, 5)

In [18]:
steps.drop_duplicates(inplace=True)

In [19]:
steps.head()

,route_id,delivery_id,action,step_order,distance
0,r-shiinc-1011686,t-amabrt-1338732,delivery,20,2.2
1,r-shiinc-1011686,t-amabrt-1338732,pickup,1,0.0
2,r-shiinc-1011686,t-amabrt-1338977,delivery,23,26.8
3,r-shiinc-1011686,t-amabrt-1338977,pickup,7,0.0
4,r-shiinc-1011686,t-amabrt-1339015,delivery,25,1.8


In [20]:
steps.shape

(1239022, 5)

In [21]:
(steps.groupby('route_id')["delivery_id"].count() % 2 != 0).sum()

0

In [22]:
df_grouped_routes_steps = (steps.groupby('route_id')["delivery_id"].count() / 2).reset_index()
df_grouped_routes_steps.columns = ['route_id', 'deliveries_count']
df_grouped_routes_steps['deliveries_count'] = df_grouped_routes_steps['deliveries_count'].astype(int)

In [23]:
df_grouped_routes_deliveries = deliveries.groupby('route_id')["id"].count().reset_index()
df_grouped_routes_deliveries.columns = ['route_id', 'deliveries_count']
df_grouped_routes_deliveries['deliveries_count'] = df_grouped_routes_deliveries['deliveries_count'].astype(int)


In [24]:
df_grouped_routes_steps.head()

,route_id,deliveries_count
0,r-shiinc-1010782,18
1,r-shiinc-1010889,17
2,r-shiinc-1010893,12
3,r-shiinc-1010901,11
4,r-shiinc-1010927,17


In [25]:
df_grouped_routes_deliveries.head()

,route_id,deliveries_count
0,r-shiinc-1010782,9
1,r-shiinc-1010889,5
2,r-shiinc-1010893,6
3,r-shiinc-1010901,5
4,r-shiinc-1010927,3


In [26]:
routes_ids = df_grouped_routes_steps.merge(df_grouped_routes_deliveries, on=['route_id', 'deliveries_count'])['route_id'].unique()

In [27]:
deliveries = deliveries.loc[deliveries.route_id.isin(routes_ids)]

In [28]:
deliveries.shape

(296421, 26)

In [29]:
steps = steps.loc[steps.route_id.isin(routes_ids)]

In [30]:
steps.shape

(592842, 5)

In [31]:
# Show all columns in pandas
pd.set_option('display.max_columns', None)

In [32]:
deliveries

,id,route_id,state,_status,items,total_size,pickupLat,pickupLng,deliveryLat,deliveryLng,pickup_dt,max_pickup_dt,pickup_arrival_dt,pickup_effective_dt,delivery_dt,max_delivery_dt,delivery_arrival_dt,delivery_effective_dt,cost,final_price,distance,shipper_id,distance_charged,distance_route,total_package_price,vehicle_capacity
25,t-amabrt-1338407,r-shiinc-1011690,7,completed,"[{""id"":""SFYBR003432117"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.6969,-43.9239,2022-01-03 12:30:00,2022-01-03 13:30:00,2022-01-03 12:43:45,2022-01-03 13:05:49,2022-01-03 22:59:26,2022-01-03 23:59:26,2022-01-03 17:19:49,2022-01-03 17:27:06,5.88,6.75,39.0,347061.0,0.8,8.1,162.2,2
66,t-amabrt-1338448,r-shiinc-1011690,7,completed,"[{""id"":""SFYBR003426195"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.6886,-43.9181,2022-01-03 12:30:00,2022-01-03 13:30:00,2022-01-03 12:43:45,2022-01-03 13:05:49,2022-01-04 00:03:21,2022-01-04 01:03:21,2022-01-03 17:05:59,2022-01-03 17:07:19,5.88,5.76,40.0,347061.0,1.0,1.0,6.6,2
74,t-amabrt-1338457,r-shiinc-1011696,7,completed,"[{""id"":""SFYBR003431274"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.7747,-44.0223,2022-01-03 15:30:00,2022-01-03 16:30:00,2022-01-03 15:10:26,2022-01-03 15:18:13,2022-01-03 20:35:57,2022-01-03 21:35:57,2022-01-03 20:38:25,2022-01-03 20:42:10,6.44,5.76,27.9,285111.0,0.0,0.0,50.9,2
98,t-amabrt-1338481,r-shiinc-1011696,7,completed,"[{""id"":""SFYBR003426292"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-19.7969,-44.0205,2022-01-03 15:30:00,2022-01-03 16:30:00,2022-01-03 15:10:26,2022-01-03 15:18:13,2022-01-03 16:38:41,2022-01-03 17:38:41,2022-01-03 17:50:21,2022-01-03 17:52:04,6.44,7.97,24.9,285111.0,0.5,0.5,148.8,2
158,t-amabrt-1338543,r-shiinc-1013661,7,completed,"[{""id"":""SFYBR003434999"",""qty"":1,""name"":""CAIXA""...",2,-19.9618,-44.0285,-20.5070,-43.8743,2022-01-04 14:00:00,2022-01-04 15:00:00,2022-01-04 14:00:40,2022-01-04 14:22:16,2022-01-04 17:42:20,2022-01-04 18:42:20,2022-01-04 23:44:56,2022-01-04 23:45:16,14.91,5.76,80.9,228334.0,0.0,0.0,13.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594263,t-wesing-3690,r-shiinc-1198145,7,completed,"[{""id"":""0"",""name"":""Marcia campos"",""size"":4,""qt...",4,-19.9743,-43.9446,-19.9603,-43.9482,2022-04-04 13:00:00,2022-04-04 14:00:00,2022-04-04 13:14:34,2022-04-04 13:25:22,2022-04-04 14:00:00,2022-04-04 15:00:00,2022-04-04 13:39:13,2022-04-04 13:44:14,6.05,12.30,4.0,74788.0,4.0,4.0,NaN,4
594264,t-wesing-3692,r-shiinc-1198145,7,completed,"[{""id"":""0"",""name"":""Viviane de Quadros"",""size"":...",5,-19.9743,-43.9446,-19.9485,-43.9439,2022-04-04 13:00:00,2022-04-04 14:00:00,2022-04-04 13:14:34,2022-04-04 13:25:22,2022-04-04 14:00:02,2022-04-04 15:00:02,2022-04-04 13:50:02,2022-04-04 13:55:30,6.05,12.30,5.5,74788.0,0.0,0.0,NaN,5
594265,t-wesing-3697,r-shiinc-1199512,7,completed,"[{""id"":""0"",""name"":""Allan Costa"",""size"":2,""qty""...",2,-19.9412,-43.9340,-19.9390,-43.9396,2022-04-04 14:45:00,2022-04-04 15:45:00,2022-04-04 14:31:48,2022-04-04 14:36:20,2022-04-04 15:45:01,2022-04-04 16:45:01,2022-04-04 14:45:41,2022-04-04 14:48:20,5.85,12.30,1.2,319781.0,1.7,1.7,NaN,2
594266,t-wesing-3698,r-shiinc-1199512,7,completed,"[{""id"":""0"",""name"":""Lucia Helena Souza de Toled...",5,-19.9412,-43.9340,-19.9355,-43.9267,2022-04-04 14:45:00,2022-04-04 15:45:00,2022-04-04 14:31:48,2022-04-04 14:36:20,2022-04-04 15:45:00,2022-04-04 16:45:00,2022-04-04 15:20:17,2022-04-04 15:21:29,5.85,12.30,2.2,319781.0,2.2,2.2,NaN,5


In [33]:
steps = steps.merge(
  deliveries[[
    'id', 'total_size', 'pickup_effective_dt',
    'delivery_effective_dt', 'cost', 'final_price',
    'shipper_id', 'pickupLat', 'pickupLng', 'deliveryLat',
    'deliveryLng'
  ]],
  left_on='delivery_id',
  right_on='id',
  how='left'
)

In [34]:
steps.head()

,route_id,delivery_id,action,step_order,distance,id,total_size,pickup_effective_dt,delivery_effective_dt,cost,final_price,shipper_id,pickupLat,pickupLng,deliveryLat,deliveryLng
0,r-shiinc-1011688,t-amabrt-1338553,delivery,41,1.5,t-amabrt-1338553,2,2022-01-03 13:08:41,2022-01-03 20:03:40,13.61,7.97,228334.0,-19.9618,-44.0285,-20.5076,-43.8763
1,r-shiinc-1011688,t-amabrt-1338553,pickup,8,0.0,t-amabrt-1338553,2,2022-01-03 13:08:41,2022-01-03 20:03:40,13.61,7.97,228334.0,-19.9618,-44.0285,-20.5076,-43.8763
2,r-shiinc-1011688,t-amabrt-1338760,delivery,29,7.9,t-amabrt-1338760,2,2022-01-03 13:08:41,2022-01-03 16:31:36,13.61,5.76,228334.0,-19.9618,-44.0285,-20.5196,-43.8271
3,r-shiinc-1011688,t-amabrt-1338760,pickup,6,0.0,t-amabrt-1338760,2,2022-01-03 13:08:41,2022-01-03 16:31:36,13.61,5.76,228334.0,-19.9618,-44.0285,-20.5196,-43.8271
4,r-shiinc-1011688,t-amabrt-1338792,delivery,35,1.9,t-amabrt-1338792,2,2022-01-03 13:08:41,2022-01-03 17:56:12,13.61,5.94,228334.0,-19.9618,-44.0285,-20.4969,-43.8583


In [35]:
def get_activityType_data(step):
  if step.action == 'pickup':
    return step['pickup_effective_dt'], step['pickupLat'], step['pickupLng']
  else:
    return step['delivery_effective_dt'], step['deliveryLat'], step['deliveryLng']

steps['effective_dt'], steps['lat'], steps['lng'] = zip(*steps.apply(get_activityType_data, axis=1))

In [36]:
path_shippers='../../data/data2/raw_2/shippers.csv'

In [37]:
shippers = pd.read_csv(path_shippers)

In [38]:
steps = steps.merge(shippers, left_on='shipper_id', right_on='id', how='left')

In [39]:
steps.head()

,route_id,delivery_id,action,step_order,distance,id_x,total_size,pickup_effective_dt,delivery_effective_dt,cost,final_price,shipper_id,pickupLat,pickupLng,deliveryLat,deliveryLng,effective_dt,lat,lng,id_y,capacity,deliveries_completed
0,r-shiinc-1011688,t-amabrt-1338553,delivery,41,1.5,t-amabrt-1338553,2,2022-01-03 13:08:41,2022-01-03 20:03:40,13.61,7.97,228334.0,-19.9618,-44.0285,-20.5076,-43.8763,2022-01-03 20:03:40,-20.5076,-43.8763,228334.0,3.0,9189.0
1,r-shiinc-1011688,t-amabrt-1338553,pickup,8,0.0,t-amabrt-1338553,2,2022-01-03 13:08:41,2022-01-03 20:03:40,13.61,7.97,228334.0,-19.9618,-44.0285,-20.5076,-43.8763,2022-01-03 13:08:41,-19.9618,-44.0285,228334.0,3.0,9189.0
2,r-shiinc-1011688,t-amabrt-1338760,delivery,29,7.9,t-amabrt-1338760,2,2022-01-03 13:08:41,2022-01-03 16:31:36,13.61,5.76,228334.0,-19.9618,-44.0285,-20.5196,-43.8271,2022-01-03 16:31:36,-20.5196,-43.8271,228334.0,3.0,9189.0
3,r-shiinc-1011688,t-amabrt-1338760,pickup,6,0.0,t-amabrt-1338760,2,2022-01-03 13:08:41,2022-01-03 16:31:36,13.61,5.76,228334.0,-19.9618,-44.0285,-20.5196,-43.8271,2022-01-03 13:08:41,-19.9618,-44.0285,228334.0,3.0,9189.0
4,r-shiinc-1011688,t-amabrt-1338792,delivery,35,1.9,t-amabrt-1338792,2,2022-01-03 13:08:41,2022-01-03 17:56:12,13.61,5.94,228334.0,-19.9618,-44.0285,-20.4969,-43.8583,2022-01-03 17:56:12,-20.4969,-43.8583,228334.0,3.0,9189.0


In [40]:
route_deliveries_count = (steps.groupby('route_id')['delivery_id'].count() / 2).reset_index()

In [41]:
steps = steps.merge(route_deliveries_count, on='route_id', how='left')

In [42]:
steps

,route_id,delivery_id_x,action,step_order,distance,id_x,total_size,pickup_effective_dt,delivery_effective_dt,cost,final_price,shipper_id,pickupLat,pickupLng,deliveryLat,deliveryLng,effective_dt,lat,lng,id_y,capacity,deliveries_completed,delivery_id_y
0,r-shiinc-1011688,t-amabrt-1338553,delivery,41,1.5,t-amabrt-1338553,2,2022-01-03 13:08:41,2022-01-03 20:03:40,13.61,7.97,228334.0,-19.9618,-44.0285,-20.5076,-43.8763,2022-01-03 20:03:40,-20.5076,-43.8763,228334.0,3.0,9189.0,22.0
1,r-shiinc-1011688,t-amabrt-1338553,pickup,8,0.0,t-amabrt-1338553,2,2022-01-03 13:08:41,2022-01-03 20:03:40,13.61,7.97,228334.0,-19.9618,-44.0285,-20.5076,-43.8763,2022-01-03 13:08:41,-19.9618,-44.0285,228334.0,3.0,9189.0,22.0
2,r-shiinc-1011688,t-amabrt-1338760,delivery,29,7.9,t-amabrt-1338760,2,2022-01-03 13:08:41,2022-01-03 16:31:36,13.61,5.76,228334.0,-19.9618,-44.0285,-20.5196,-43.8271,2022-01-03 16:31:36,-20.5196,-43.8271,228334.0,3.0,9189.0,22.0
3,r-shiinc-1011688,t-amabrt-1338760,pickup,6,0.0,t-amabrt-1338760,2,2022-01-03 13:08:41,2022-01-03 16:31:36,13.61,5.76,228334.0,-19.9618,-44.0285,-20.5196,-43.8271,2022-01-03 13:08:41,-19.9618,-44.0285,228334.0,3.0,9189.0,22.0
4,r-shiinc-1011688,t-amabrt-1338792,delivery,35,1.9,t-amabrt-1338792,2,2022-01-03 13:08:41,2022-01-03 17:56:12,13.61,5.94,228334.0,-19.9618,-44.0285,-20.4969,-43.8583,2022-01-03 17:56:12,-20.4969,-43.8583,228334.0,3.0,9189.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592837,r-shiinc-1273881,t-tagros-2545511,pickup,2,0.0,t-tagros-2545511,2,2022-05-17 15:56:51,2022-05-17 17:08:44,9.44,5.95,127284.0,-19.9618,-44.0286,-19.9478,-43.9404,2022-05-17 15:56:51,-19.9618,-44.0286,127284.0,3.0,8.0,3.0
592838,r-shiinc-1273881,t-tagros-2548135,delivery,6,1.2,t-tagros-2548135,2,2022-05-17 15:56:51,2022-05-17 17:16:44,9.44,5.95,127284.0,-19.9618,-44.0286,-19.9488,-43.9343,2022-05-17 17:16:44,-19.9488,-43.9343,127284.0,3.0,8.0,3.0
592839,r-shiinc-1273881,t-tagros-2548135,pickup,1,0.0,t-tagros-2548135,2,2022-05-17 15:56:51,2022-05-17 17:16:44,9.44,5.95,127284.0,-19.9618,-44.0286,-19.9488,-43.9343,2022-05-17 15:56:51,-19.9618,-44.0286,127284.0,3.0,8.0,3.0
592840,r-shiinc-1273881,t-tagros-2548918,delivery,4,13.8,t-tagros-2548918,2,2022-05-17 15:56:51,2022-05-17 17:07:43,9.44,5.95,127284.0,-19.9618,-44.0286,-19.9483,-43.9425,2022-05-17 17:07:43,-19.9483,-43.9425,127284.0,3.0,8.0,3.0


In [43]:
steps.drop(columns=['id_x', 'id_y', 'pickup_effective_dt', 'delivery_effective_dt', 'pickupLat', 'pickupLng', 'deliveryLat', 'deliveryLng'], inplace=True)

In [44]:
steps.rename(
  columns={
    'capacity': 'shipper_capacity',
    'deliveries_completed': 'shipper_deliveries_completed',
    'delivery_id_x': 'delivery_id',
    'delivery_id_y': 'route_deliveries_count'
  },
  inplace=True
)

In [45]:
steps.head()

,route_id,delivery_id,action,step_order,distance,total_size,cost,final_price,shipper_id,effective_dt,lat,lng,shipper_capacity,shipper_deliveries_completed,route_deliveries_count
0,r-shiinc-1011688,t-amabrt-1338553,delivery,41,1.5,2,13.61,7.97,228334.0,2022-01-03 20:03:40,-20.5076,-43.8763,3.0,9189.0,22.0
1,r-shiinc-1011688,t-amabrt-1338553,pickup,8,0.0,2,13.61,7.97,228334.0,2022-01-03 13:08:41,-19.9618,-44.0285,3.0,9189.0,22.0
2,r-shiinc-1011688,t-amabrt-1338760,delivery,29,7.9,2,13.61,5.76,228334.0,2022-01-03 16:31:36,-20.5196,-43.8271,3.0,9189.0,22.0
3,r-shiinc-1011688,t-amabrt-1338760,pickup,6,0.0,2,13.61,5.76,228334.0,2022-01-03 13:08:41,-19.9618,-44.0285,3.0,9189.0,22.0
4,r-shiinc-1011688,t-amabrt-1338792,delivery,35,1.9,2,13.61,5.94,228334.0,2022-01-03 17:56:12,-20.4969,-43.8583,3.0,9189.0,22.0


In [46]:
steps.sort_values(by=['route_id', 'step_order'], inplace=True)

In [47]:
steps.reset_index(drop=True, inplace=True)

In [48]:
steps.head()

,route_id,delivery_id,action,step_order,distance,total_size,cost,final_price,shipper_id,effective_dt,lat,lng,shipper_capacity,shipper_deliveries_completed,route_deliveries_count
0,r-shiinc-1011382,t-petz-2548122,pickup,1,0.0,1,6.48,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.957,3.0,1736.0,20.0
1,r-shiinc-1011382,t-petz-2547366,pickup,2,0.0,4,6.48,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.957,3.0,1736.0,20.0
2,r-shiinc-1011382,t-petz-2546288,pickup,3,0.0,3,6.48,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.957,3.0,1736.0,20.0
3,r-shiinc-1011382,t-petz-2546397,pickup,4,0.0,1,6.48,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.957,3.0,1736.0,20.0
4,r-shiinc-1011382,t-petz-2547114,pickup,5,0.0,3,6.48,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.957,3.0,1736.0,20.0


In [49]:
def get_previous_step_data(step):
  index = step.name
  is_first_step = step.step_order == 1
  time_from_previous = 0
  if not is_first_step:
    time_from_previous = (step.effective_dt - steps.loc[index - 1, 'effective_dt']).total_seconds()
  return time_from_previous

In [50]:
steps['time_from_previous'] = steps.apply(get_previous_step_data, axis=1)
steps.rename(columns={'distance': 'distance_from_previous'}, inplace=True)

In [51]:
steps.shape

(592842, 16)

In [52]:
steps = steps.loc[steps.time_from_previous >= 0]

In [53]:
steps = steps.loc[np.abs(stats.zscore(steps['time_from_previous'])) < 3]

In [54]:
steps.shape

(512369, 16)

In [55]:
steps.route_id.nunique()

9730

In [56]:
steps.reset_index(drop=True, inplace=True)

In [57]:
steps

,route_id,delivery_id,action,step_order,distance_from_previous,total_size,cost,final_price,shipper_id,effective_dt,lat,lng,shipper_capacity,shipper_deliveries_completed,route_deliveries_count,time_from_previous
0,r-shiinc-1011382,t-petz-2548122,pickup,1,0.0,1,6.4800,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.9570,3.0,1736.0,20.0,0.0
1,r-shiinc-1011382,t-petz-2547366,pickup,2,0.0,4,6.4800,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.9570,3.0,1736.0,20.0,0.0
2,r-shiinc-1011382,t-petz-2546288,pickup,3,0.0,3,6.4800,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.9570,3.0,1736.0,20.0,0.0
3,r-shiinc-1011382,t-petz-2546397,pickup,4,0.0,1,6.4800,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.9570,3.0,1736.0,20.0,0.0
4,r-shiinc-1011382,t-petz-2547114,pickup,5,0.0,3,6.4800,10.0,170162.0,2022-01-03 12:30:51,-19.9627,-43.9570,3.0,1736.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512364,r-shiinc-1353194,t-petz-3430038,pickup,2,0.0,3,8.4525,11.3,170749.0,2022-07-01 18:34:18,-19.9627,-43.9570,3.0,23294.0,3.0,0.0
512365,r-shiinc-1353194,t-petz-3430470,pickup,3,0.0,1,8.4525,11.3,170749.0,2022-07-01 18:34:18,-19.9627,-43.9570,3.0,23294.0,3.0,0.0
512366,r-shiinc-1353194,t-petz-3430169,delivery,4,4.8,4,8.4525,11.3,170749.0,2022-07-01 19:39:39,-19.9429,-43.9662,3.0,23294.0,3.0,3921.0
512367,r-shiinc-1353194,t-petz-3430038,delivery,5,3.8,3,8.4525,11.3,170749.0,2022-07-01 20:31:48,-19.9219,-43.9784,3.0,23294.0,3.0,3129.0


In [60]:
path_to_save='../../data/data2/processed/'

In [61]:
steps.to_csv(f'{path_to_save}steps_2.csv', index=False)